In [5]:
#Task 2
# Import necessary libraries
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Define the MultiTaskBERT model
class MultiTaskBERT(nn.Module):
    def __init__(self, model_name, num_classes_task1, num_classes_task2):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)  # Load pre-trained BERT model
        self.dropout = nn.Dropout(0.3)  # Apply dropout
        # Task 1 classifier: output dimension = num_classes_task1
        self.classifier_task1 = nn.Linear(self.bert.config.hidden_size, num_classes_task1)
        # Task 2 classifier: output dimension = num_classes_task2
        self.classifier_task2 = nn.Linear(self.bert.config.hidden_size, num_classes_task2)

    def forward(self, input_ids, attention_mask):
        # Feed input to BERT model
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Get pooled output
        pooled_output = self.dropout(pooled_output)  # Apply dropout

        # Task 1 classification
        task1_logits = self.classifier_task1(pooled_output)
        # Task 2 classification
        task2_logits = self.classifier_task2(pooled_output)

        return task1_logits, task2_logits

# Data preparation
# Sample sentences and labels
sentences = [
    "I love this movie!", "This is the worst film.", "It was an average experience.",
    "Absolutely fantastic!", "Terrible, I hated it.", "Just okay, not great.",
    "Brilliant work!", "Not good, very disappointing.", "Mediocre at best.",
    "A masterpiece!", "Awful movie.", "Decent watch.",
    "Amazing storyline!", "I would not recommend it.", "It was a good watch.",
    "Fantastic film!", "Horrible experience.", "Quite boring.",
    "Wonderful acting!", "Poorly executed.", "Nothing special.",
    "Stellar performance!", "Waste of time.", "Enjoyed it a lot."
]
# Labels for task 1 (sentiment classification)
labels_task1 = ["positive", "negative", "neutral", "positive", "negative", "neutral",
                "positive", "negative", "neutral", "positive", "negative", "neutral",
                "positive", "negative", "neutral", "positive", "negative", "neutral",
                "positive", "negative", "neutral", "positive", "negative", "neutral"]
# Labels for task 2 (classification)
labels_task2 = [2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1]

# Tokenize input sentences
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Encode labels
le_task1 = LabelEncoder()
labels_task1 = le_task1.fit_transform(labels_task1)
labels_task1 = torch.tensor(labels_task1)
labels_task2 = torch.tensor(labels_task2)

# Create dataset and dataloader
dataset = TensorDataset(inputs.input_ids, inputs.attention_mask, labels_task1, labels_task2)
dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=2)

# Hyperparameters
learning_rate = 2e-5
batch_size = 16
num_epochs = 20

# Model initialization
model_name = "bert-base-uncased"
model = MultiTaskBERT(model_name, num_classes_task1=3, num_classes_task2=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0.0

    for batch in dataloader:
        input_ids, attention_mask, labels_task1, labels_task2 = batch

        optimizer.zero_grad()
        logits_task1, logits_task2 = model(input_ids, attention_mask)

        loss_task1 = nn.CrossEntropyLoss()(logits_task1, labels_task1)
        loss_task2 = nn.CrossEntropyLoss()(logits_task2, labels_task2)
        loss = loss_task1 + loss_task2

        loss.backward()
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

    # Print epoch loss after each epoch
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}")

# Evaluation and inference
model.eval()
test_sentences = [
    "The movie was incredible!", "Very worst direction", "Acting was just normal.","I regret watching it.", "Awesome screenplay","Not bad,but not good either"
]

with torch.no_grad():
    for test_sentence in test_sentences:
        inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
        input_ids, attention_mask = inputs.input_ids, inputs.attention_mask

        logits_task1, logits_task2 = model(input_ids, attention_mask)
        pred_task1 = torch.argmax(logits_task1, dim=1).item()
        pred_task2 = torch.argmax(logits_task2, dim=1).item()

        print(f"Review: {test_sentence}")
        print(f"Task A Prediction: {le_task1.inverse_transform([pred_task1])[0]}")
        print(f"Task B Prediction: {pred_task2}\n")


Epoch 1, Loss: 1.9357398947079976
Epoch 2, Loss: 1.4068276484807332
Epoch 3, Loss: 1.2356445342302322
Epoch 4, Loss: 1.1329237719376881
Epoch 5, Loss: 0.9479951411485672
Epoch 6, Loss: 0.8400726169347763
Epoch 7, Loss: 0.5782277062535286
Epoch 8, Loss: 0.537917748093605
Epoch 9, Loss: 0.37937616060177487
Epoch 10, Loss: 0.34428155918916065
Epoch 11, Loss: 0.2711183937887351
Epoch 12, Loss: 0.22544304033120474
Epoch 13, Loss: 0.20548463240265846
Epoch 14, Loss: 0.17774531245231628
Epoch 15, Loss: 0.16182213400801024
Epoch 16, Loss: 0.15414701774716377
Epoch 17, Loss: 0.15711895003914833
Epoch 18, Loss: 0.14500237132112184
Epoch 19, Loss: 0.15142372560997805
Epoch 20, Loss: 0.15061706863343716
Review: The movie was incredible!
Task A Prediction: positive
Task B Prediction: 2

Review: Very worst direction
Task A Prediction: negative
Task B Prediction: 0

Review: Acting was just normal.
Task A Prediction: neutral
Task B Prediction: 1

Review: I regret watching it.
Task A Prediction: negati